<a href="https://colab.research.google.com/github/jhhan0/learning-nlp/blob/main/cosine_similarity_movies_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
from google.colab import files
files.upload()

Saving movies_metadata.csv to movies_metadata.csv


In [ ]:
data = pd.read_csv('movies_metadata.csv', low_memory=False)
data = data.head(20000)
data.tail()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19995,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",http://www.lordreetlamorale-lefilm.com/#/nav/t...,76609,tt1242521,fr,L'Ordre et la Morale,Dissidents in a French colony attack a police ...,1.59284,/uuyCdlOYcqvSYkKNp3hFBQ5oay2.jpg,"[{'name': 'France 2 Cinéma', 'id': 83}, {'name...","[{'iso_3166_1': 'FR', 'name': 'France'}]",2011-11-16,0.0,136.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Rebellion,False,6.3,24.0
19996,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,8941,tt1132607,fr,Versailles,A young mother Nina and her son Enzo find them...,0.705023,/dYViK2Ymxeh0VHggeHIXKdydoST.jpg,"[{'name': 'Angoa-Agicoa', 'id': 309}, {'name':...","[{'iso_3166_1': 'FR', 'name': 'France'}]",2008-05-19,0.0,113.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Versailles,False,6.0,4.0
19997,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}, {'id': 107...",NaN,64074,tt1528224,en,Two in the Wave,An in-depth analysis of the relationship betwe...,0.232855,/hltX0E33tlBnMpjSTcrSv60eeIq.jpg,[],[],2010-05-19,0.0,91.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,NaN,Two in the Wave,False,5.5,5.0
19998,False,NaN,0,[],NaN,197723,tt0471239,en,Lotte Reiniger: Homage to the Inventor of the ...,Follows the life and work of animator Lotte Re...,0.002353,NaN,[],[],2001-11-19,0.0,0.0,[],Released,NaN,Lotte Reiniger: Homage to the Inventor of the ...,False,0.0,0.0
19999,False,NaN,0,[],NaN,269797,tt0492337,en,"RKO Production 601: The Making of 'Kong, the E...","An in-depth look at the genesis, production, a...",0.289677,/lMOrGkhsFHopn1STpVCizZVNmVa.jpg,[],[],2005-10-23,0.0,0.0,[],Released,NaN,"RKO Production 601: The Making of 'Kong, the E...",False,8.0,1.0


In [ ]:
# If column 'overview" contains NaN (missing) value(s), change it in order to process TF-IDF vectorization
data['overview'] = data['overview'].fillna('')
data['overview'].isnull().sum()

0

In [ ]:
# TF-IDF processing
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

(20000, 47487)


In [ ]:
# Get cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print(cosine_sim.shape)

(20000, 20000)


In [ ]:
# Create table that contains movie title and its index
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head(5))

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get a index of corresponding title name
    index = indices[title]
    sim_scores = list(enumerate(cosine_sim[index]))
    # Sort sim_scores list from the highest to lowest cosine similarity
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the top 10 similar movies (highest cosine similarity (it means cosine similarity close to 1)), but except itself 
    sim_scores = sim_scores[1:11]
    # Create movie_indices list to store top 10 movies' index
    movie_indices = [i[0] for i in sim_scores]
    return data['title'].iloc[movie_indices]

In [ ]:
get_recommendations('Superman')

16626                            All Star Superman
2527                                   Superman II
9169     The Batman Superman Movie: World's Finest
11067                             Superman Returns
2529              Superman IV: The Quest for Peace
17852                    Superman and the Mole-Men
19741                   Gable: The King Remembered
6822                                  The Freshman
12617                          The Ron Clark Story
19063              Superman/Batman: Public Enemies
Name: title, dtype: object